<a href="https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這支 notebook 是為了選取剪輯 news-digest 的部分新聞分類，以利後續轉貼
執行步驟如下：
1. 按下 Install cell 前面的 ► 按鈕來安裝 clip 模組.
2. 根據需要，填表執行後面的 Forms 或 Widgets UI

### Install 

In [11]:
!curl -o clip.py https://raw.githubusercontent.com/YorkJong/news-digest/main/src/clip.py
from clip import *

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4967  100  4967    0     0  26561      0 --:--:-- --:--:-- --:--:-- 26561


### Select Categories with the Colab Forms

In [15]:
#@title Single Selection { run: "auto" }
category = "Finance" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = True #@param {type:"boolean"}

fn = get_latest_fn("journals")
content = get_file(fn)
lines = get_lines_of_categories([category], content, with_hashtags, with_headers)
print('\n'.join(lines))

# Copy text to clipboard in Python using pandas module
#import pandas as pd
#df=pd.DataFrame(['Text to copy'])
#df.to_clipboard(index=False, header=False)

### Finance
- [麥可貝瑞分析銀行暴雷：瑞銀快落入 SVB 所在象限](https://finance.technews.tw/2023/03/21/michael-burry-svb-ubs-group-ag/)
- [美國金融業吵著要糖！中小銀行聯盟要FDIC「給全部存款保險2年」避免擠兌爆發](https://www.blocktempo.com/us-midsize-banks-seek-fdic-insurance/)
- [一夜崩盤 多次熔斷 股價腰斬 下一家破產的美國銀行？](https://news.cnyes.com/news/id/5117564)
- [富爸爸預言全中！瑞信被收購、黃金突破2000鎂…再砲轟聯準會「通膨大騙子」](https://www.blocktempo.com/kiyosaki-warns-u-s-inflation-is-now-systemic/)
- [銀行體系承壓 高盛：Fed本周將暫停升息](https://news.cnyes.com/news/id/5117507)
- [FDIC將分拆矽谷銀行 尋求個別拍賣其私人銀行部門](https://news.cnyes.com/news/id/5117513)
- [FT：中國改變支持晶片產業方式 讓大咖更易獲補貼](https://news.cnyes.com/news/id/5117643)
-


In [5]:
#@title Multiple Selections { run: "auto" }
first_category = "Tesla & SpaceX; Vehicle" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
last_category = "Finance" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = True #@param {type:"boolean"}

fn = get_latest_fn("journals")
content = get_file(fn)
categories = get_categories(content)
categories = get_sublist(categories, first_category, last_category)
lines = get_lines_of_categories(categories, content, with_hashtags, with_headers)
print('\n'.join(lines))


### Tesla & SpaceX; Vehicle
- [[U-EV]Tesla率先在瑞典設立無人遠端試駕中心，無接觸試駕體驗擴展至歐洲](https://news.u-car.com.tw/news/article/74334)
- [英皇國際與Tesla合作,旗下5商場增設V3超級充電站](https://news.cnyes.com/news/id/5119852)
- [英皇國際及Tesla簽署「綠色運輸合作備忘錄」共建智慧未來](https://today.line.me/hk/v2/article/BEP8jOQ)
- [特斯拉信評脫離垃圾級進入投資級 穆迪跟進標普之後調升](https://udn.com/news/story/6811/7045090)
- [特斯拉申請將德國超級工廠年產能翻倍至100萬輛](https://news.cnyes.com/news/id/5120152)
- [通膨削減法案刺激北美製電動車銷量大增，特斯拉週一股價漲1.73%](https://tw.stock.yahoo.com/news/通膨削減法案刺激北美製電動車銷量大增-特斯拉週-股價漲1-73-004514463.html)
- [FOMC會議今晚登場、穆迪調升特斯拉信評！](https://news.cnyes.com/news/id/5121144)
- [豐田宣布引進Mirai氫能車！預告現身台灣智慧移動展-國內車訊|8891汽車](https://c.8891.com.tw/news/16578)
- [賓士將砸數十億美元，投資電動車廠](https://technews.tw/2023/03/20/benz-to-spend-billions-investing-in-electric-car-factory/)
- [麥拉倫切入電動車產業，但不是靠車子](https://finance.technews.tw/2023/03/20/mclaren-inverter-ev-industry/)
-
### Tech Titans
- [NVIDIA成功不靠幸運黃仁勳喊話：AI時代快買A100顯卡](https://news.xfastest.com/nvidia/125576/nvidia-43/)
- [微軟子公司Nuance要以GPT-4來產

### Select multiple categories with Jupyter Widgets

In [6]:
import ipywidgets as widgets

fn = get_latest_fn("journals")
content = get_file(fn)
options = get_categories(content)

categories = widgets.SelectMultiple(
    options=options,
    value=[options[0]],
    rows=len(options),
    description='Categories',
    disabled=False
)

with_hashtags = widgets.Checkbox(
    value=True,
    description='with hashtags',
    disabled=False
)

with_headers = widgets.Checkbox(
    value=True,
    description='with headers',
    disabled=False
)

flags = widgets.VBox([with_hashtags, with_headers])
widgets.HBox([categories, flags])

In [13]:
lines = get_lines_of_categories(categories.value, content, with_hashtags.value, with_headers.value)
print('\n'.join(lines))

### Finance
- [麥可貝瑞分析銀行暴雷：瑞銀快落入 SVB 所在象限](https://finance.technews.tw/2023/03/21/michael-burry-svb-ubs-group-ag/)
- [美國金融業吵著要糖！中小銀行聯盟要FDIC「給全部存款保險2年」避免擠兌爆發](https://www.blocktempo.com/us-midsize-banks-seek-fdic-insurance/)
- [一夜崩盤 多次熔斷 股價腰斬 下一家破產的美國銀行？](https://news.cnyes.com/news/id/5117564)
- [富爸爸預言全中！瑞信被收購、黃金突破2000鎂…再砲轟聯準會「通膨大騙子」](https://www.blocktempo.com/kiyosaki-warns-u-s-inflation-is-now-systemic/)
- [銀行體系承壓 高盛：Fed本周將暫停升息](https://news.cnyes.com/news/id/5117507)
- [FDIC將分拆矽谷銀行 尋求個別拍賣其私人銀行部門](https://news.cnyes.com/news/id/5117513)
- [FT：中國改變支持晶片產業方式 讓大咖更易獲補貼](https://news.cnyes.com/news/id/5117643)
-
